In [48]:
import seaborn as sns
import pandas as pd

In [49]:
df_base = pd.read_excel('PCP.xlsx',sheet_name='Base')
df_base.columns

Index(['Projeto', 'Núcleo', 'Portfólio', 'Tipo', 'Valor',
       'Duração no contrato (semanas)', 'Nº Consultores', 'Previsão de Início',
       'Fim Contratual', 'Equipe', 'Início Real', 'Semanas OFF',
       'Fim Previsto', 'Fim Real', 'Status', 'TEP', 'Aditivo?',
       'CSAT coletado?', 'NPS de Experiência', 'NPS de Resultado', 'ENB',
       'Coordenador/validador', 'Analista 1', 'Analista 2', 'Analista 3',
       'Analista 4', 'Semanas x Consultores', 'Receita perdida', 'Atrasado?',
       'Atraso', 'Início Real - início previsto',
       'Data p/ coletar NPS de resultado', 'NPS da Conceps'],
      dtype='object')

In [50]:
def get_analistas(df_base):
    analysts_columns = ['Analista 2', 'Analista 3', 'Analista 4']
    unique_analysts = pd.unique(df_base[analysts_columns].values.ravel('K'))

    unique_analysts = unique_analysts[~pd.isna(unique_analysts)]  
    analysts_df = pd.DataFrame(unique_analysts, columns=['Analista'])
    nucleus_mapping = {}
    for column in analysts_columns:
        for idx, row in df_base.iterrows():
            analyst = row[column]
            if pd.notna(analyst) and analyst not in nucleus_mapping:
                nucleus_mapping[analyst] = row['Núcleo']

    # Add the nucleus information to the analysts dataframe
    analysts_df['Núcleo'] = analysts_df['Analista'].map(nucleus_mapping)
    return analysts_df


In [51]:
base_analista = get_analistas(df_base)

In [52]:
base_analista.head(30)

,Analista,Núcleo
0,robson.sousa,NCiv
1,guilherme.nunes,NCiv
2,bianca.vilasboas,NCiv
3,guilherme.mendes,NCiv
4,joaopedro.brunheroto,NCiv
5,nathalia.valle,NCiv
6,miguel.victor,NCiv
7,gabriela.fernandes,NCiv
8,milena.ramos,NCiv
9,anaclara.pacifico,NCon


In [53]:
def generate_pcp(df_base,base_analista):
    # Convert the columns to datetime, ignoring errors
    df_base['Início Real'] = pd.to_datetime(df_base['Início Real'], errors='coerce')
    df_base['Fim Previsto'] = pd.to_datetime(df_base['Fim Previsto'], errors='coerce')

    # Find the earliest start date and the latest end date
    start_date = df_base['Início Real'].min()
    end_date = df_base['Fim Previsto'].max()

    # Generate a DataFrame for each day between the start and end dates
    pacing_dates = pd.date_range(start=start_date, end=end_date, freq='D')
    pacing_table = pd.DataFrame(pacing_dates, columns=['Data'])
        # Filter projects with status 'Executando'
    # Filter projects with status 'Executando' again if needed
    executing_projects = df_base[df_base['Status'] == 'Executando']

    # Initialize the new column for project-analyst mapping
    pacing_table['Projeto por Analista'] = [{} for _ in range(len(pacing_table))]

    # Create a mapping for each day with the projects and corresponding analysts
    analyst_columns = ['Analista 1','Analista 2', 'Analista 3', 'Analista 4']

    # Iterate over each executing project
    for _, project in executing_projects.iterrows():
        start = project['Início Real']
        end = project['Fim Previsto']
        project_name = project['Projeto']
        if pd.notna(start) and pd.notna(end) and end >= start:
            # Identify all analysts involved in the project
            involved_analysts = project[analyst_columns].dropna().unique()
            # For each day the project is running, add the project to the corresponding analysts
            for day in pacing_table.loc[(pacing_table['Data'] >= start) & (pacing_table['Data'] <= end), 'Data']:
                for analyst in involved_analysts:
                    if analyst in pacing_table.loc[pacing_table['Data'] == day, 'Projeto por Analista'].values[0]:
                        pacing_table.loc[pacing_table['Data'] == day, 'Projeto por Analista'].values[0][analyst].append(project_name)
                    else:
                        pacing_table.loc[pacing_table['Data'] == day, 'Projeto por Analista'].values[0][analyst] = [project_name]

            # Display the updated table to confirm changes
    return pacing_table

# Calculate the number of projects per analyst for each day
def calculate_projects_per_analyst(daily_dict):
        if not daily_dict:
            return 0  # Return 0 if no projects or analysts are active that day
        all_projects = set()
        all_analysts = set()
        
        for analyst, projects in daily_dict.items():
            all_analysts.add(analyst)
            all_projects.update(projects)
        
        if len(all_analysts) == 0:
            return 0  # Avoid division by zero
        return len(all_projects) / len(all_analysts)





In [54]:
pacing_pcp = generate_pcp(df_base,base_analista)
# Apply the function to each day's dictionary of projects and analysts
pacing_pcp['Projetos por Analista'] = pacing_pcp['execucao'].apply(calculate_projects_per_analyst)

KeyError: 'execucao'

In [ ]:
pacing_pcp.head()

,Data,Projeto por Analista,Projetos por Analista
0,2023-01-29,"{'gabriel.saad': ['J&A'], 'joaopedro.brunherot...",0.333333
1,2023-01-30,"{'gabriel.saad': ['J&A'], 'joaopedro.brunherot...",0.333333
2,2023-01-31,"{'gabriel.saad': ['J&A'], 'joaopedro.brunherot...",0.333333
3,2023-02-01,"{'gabriel.saad': ['J&A'], 'joaopedro.brunherot...",0.333333
4,2023-02-02,"{'gabriel.saad': ['J&A'], 'joaopedro.brunherot...",0.333333
